### Reviews about CDG

In this notebook, we are going to perform topic modelling based on sentiments from various review sites. 

In [1]:
# Base
import os, re, string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

# NLTK
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.metrics.distance import edit_distance
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer

# Topic Modelling
import pyLDAvis
import pyLDAvis.gensim
import gensim
from gensim import corpora
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to D:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


### Data Preparation

We will first get the relevant files we need and thereafter extract them out into a list of reviews.

In [2]:
# Merge into 

path='.'

filename_sentiment_list=[]

for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Check what are the files in the folder
        print(os.path.join(path, filename))
        
        #Read the files and put to df
        df = pd.read_csv(filename,encoding = "ISO-8859-1")
        
        #Add it into a list
        try:
            filename_sentiment_list.extend(df['comment'].tolist())
            filename_sentiment_list.extend(df['tweet'].tolist())
            filename_sentiment_list.extend(df['title'].tolist())
        except:
            pass

print(filename_sentiment_list)

.\gplay_cdge.csv
.\reddit_cdgtaxi.csv
.\reddit_cdgtaxi_en_extracted.csv
.\reddit_comfortdelgro.csv
.\reddit_comfortdelgro_extracted.csv
.\twitter_cdgtaxi.csv
.\twitter_comfortdelgro.csv
['Horrible experience. waited 8mins for the cab to arrive and was cancelled stating no show. WTH.', 'Can be better. Please benchmark to Grab to get better customer satisfactory eg using credit card when ride is completed.', 'Not so good', 'Good one', 'Unable to use app. SMS OTP not delivered (Malaysia number)', 'Easy to use', 'Good app.', 'It s not easy to use', 'This app not working with Samsung S7. I try to sign in a lot of times but nobody send me verify code..', 'Great app to rely on when grab is having ridiculous price hikes', 'Good', 'Useless app without pick up point options.', "Didn't work. Did not send unlock code to my international mobile", 'After update the new version my previous saved customer address gone..only 10 cust only..I always go cust side and have been 4 year save then today lost 

In [3]:
filename_sentiment_list[5:7]

['Easy to use', 'Good app.']

### Cleaning

We will also perform cleaning

In [4]:
def clean_list_tokenise(reviews_list):
    # Tokenise the words
    tokenizer = RegexpTokenizer(r'\w+')
    return [tokenizer.tokenize(sentence) for sentence in reviews_list]

def clean_list_lemma(reviews_list):
    # Lemmatization
    lemma = WordNetLemmatizer()
    return [[lemma.lemmatize(word) for word in sentence] for sentence in reviews_list]

def clean_list_stopwords(reviews_list, stop_other=[]):
    def stopword_condition(word):
        word = word.lower()
        
        return word not in stopwords.words('english') \
            and word not in stop_other \
            and word[:5] != 'http' \
            and word[:5] != 'https' \
            and word[:2] != 'RT' \
            and word[0] != '@'
    
    return [[w for w in s if stopword_condition(w)] for s in reviews_list]

In [5]:
sentiment_filtered = clean_list_stopwords(clean_list_tokenise(filename_sentiment_list))
len(sentiment_filtered)

1193

In [6]:
sentiment_filtered_sentences = [' '.join(s) for s in sentiment_filtered]
sentiment_filtered_sentences[:5]

['Horrible experience waited 8mins cab arrive cancelled stating show WTH',
 'better Please benchmark Grab get better customer satisfactory eg using credit card ride completed',
 'good',
 'Good one',
 'Unable use app SMS OTP delivered Malaysia number']

### Segregating by Sentiment

We will segregate the reviews by sentiment. This is achieved by the `analyse_sentiment_vader` function, which uses vader to help us analyse the degree to which the particular review or comment is positive, negative or neutral. It also computes a compound score ranging from -1 to 1 which takes into account all the scores. More reading for the sentiment analysis are available at:

- https://opensourceforu.com/2016/12/analysing-sentiments-nltk/
- http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html

In [7]:
def analyse_sentiment_vader(df, col_name):
    sid = SentimentIntensityAnalyzer()
    vader = lambda text: sid.polarity_scores(text)
    
    df['vader'] = df[col_name].apply(vader)
    df = pd.merge(df, df['vader'].apply(pd.Series), left_index=True, right_index=True)
    return df.drop(['vader'], axis=1)

In [8]:
df_sentiment_filtered = pd.DataFrame(sentiment_filtered_sentences, columns=['comment'])
df_sentiment_filtered = analyse_sentiment_vader(df_sentiment_filtered, 'comment')
df_sentiment_filtered.head(10)

,comment,neg,neu,pos,compound
0,Horrible experience waited 8mins cab arrive ca...,0.579,0.421,0.000,-0.8636
1,better Please benchmark Grab get better custom...,0.000,0.405,0.595,0.9042
2,good,0.000,0.000,1.000,0.4404
3,Good one,0.000,0.256,0.744,0.4404
4,Unable use app SMS OTP delivered Malaysia number,0.000,0.843,0.157,0.0772
5,Easy use,0.000,0.256,0.744,0.4404
6,Good app,0.000,0.256,0.744,0.4404
7,easy use,0.000,0.256,0.744,0.4404
8,app working Samsung S7 try sign lot times nobo...,0.000,1.000,0.000,0.0000
9,Great app rely grab ridiculous price hikes,0.216,0.431,0.353,0.3818


In [9]:
df_sentiment_filtered.tail(10)

,comment,neg,neu,pos,compound
1183,write credit 10 account,0.000,0.536,0.464,0.3818
1184,category select writing,0.000,1.000,0.000,0.0000
1185,Anyhow wack need get access form fill,0.000,1.000,0.000,0.0000
1186,Im curious main gripes current taxi apps,0.000,0.723,0.277,0.3182
1187,using Cabbie years pretty much satisfied bad r...,0.063,0.829,0.108,0.3612
1188,advantage using app smsing directly,0.000,0.667,0.333,0.2500
1189,SGTransport mobilityandme sadly Windows Phone ...,0.049,0.820,0.131,0.6908
1190,Also recommend uber smoothest app gps works we...,0.079,0.526,0.396,0.9670
1191,wanted recommend uber app works supply enough ...,0.094,0.750,0.156,0.2500
1192,true guess thinking app much overall product u...,0.000,0.741,0.259,0.4215


In [10]:
pos_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] >= 0]['comment'])
neg_list = list(df_sentiment_filtered[df_sentiment_filtered['compound'] < 0]['comment'])

len(pos_list), len(neg_list)

(844, 349)

In [11]:
pos_list_clean = clean_list_lemma(clean_list_tokenise(pos_list))
neg_list_clean = clean_list_lemma(clean_list_tokenise(neg_list))

In [12]:
len(pos_list_clean), len(neg_list_clean)

(844, 349)

Ignore code below

### Topic Modelling

In [13]:
neg_dict = corpora.Dictionary(neg_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in neg_list_clean]

print(doc_term_matrix[0:10])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(16, 1)], [(11, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(11, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 2), (32, 2), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 3), (41, 1), (42, 1), (43, 1), (44, 1)], [(5, 1), (15, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2)], [(6, 1), (16, 1), (57, 1), (58, 1), (59, 1)], [(60, 1), (61, 1), (62, 1), (63, 1)], [(49, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(69, 1), (78, 1), (79, 1)]]


In [29]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word=neg_dict, passes=50)

In [30]:
print(ldamodel.print_topics(num_topics=2, num_words=5))

[(0, '0.028*"Uber" + 0.026*"Grab" + 0.017*"taxi" + 0.014*"driver" + 0.010*"ride"'), (1, '0.039*"app" + 0.020*"taxi" + 0.017*"booking" + 0.013*"cab" + 0.010*"use"')]


In [31]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, neg_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.109299  0.0       1        1  60.599155
1     -0.109299  0.0       2        1  39.400845, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
11    Default  169.000000         app  169.000000  30.0000  30.0000
459   Default  159.000000        Uber  159.000000  29.0000  29.0000
241   Default  146.000000        Grab  146.000000  28.0000  28.0000
49    Default   70.000000     booking   70.000000  27.0000  27.0000
306   Default   37.000000    location   37.000000  26.0000  26.0000
255   Default   31.000000       phone   31.000000  25.0000  25.0000
627   Default   35.000000       surge   35.000000  24.0000  24.0000
4     Default   65.000000         cab   65.000000  23.0000  23.0000
1101  Default   43.000000      market   43.000000  22.0000  22.0000
296   Default   21.000000          10   21.000000  21.0000  21.0000
120   Default   61.000000        ride   61.000000  20.0000  20.0000
155   Default   30.000000        book   30.000000  19.0000  19.0000
908   Default   19.000000        Good   19.000000  18.0000  18.0000
95    Default   83.000000      driver   83.000000  17.0000  17.0000
1257  Default   35.000000    business   35.000000  16.0000  16.0000
101   Default   18.000000     address   18.000000  15.0000  15.0000
851   Default   35.000000         car   35.000000  14.0000  14.0000
42    Default   57.000000         use   57.000000  13.0000  13.0000
1138  Default   39.000000   Singapore   39.000000  12.0000  12.0000
131   Default   36.000000        fare   36.000000  11.0000  11.0000
121   Default   30.000000     service   30.000000  10.0000  10.0000
149   Default   45.000000        even   45.000000   9.0000   9.0000
220   Default   20.000000        Taxi   20.000000   8.0000   8.0000
195   Default   14.000000  masterpass   14.000000   7.0000   7.0000
633   Default   17.000000     message   17.000000   6.0000   6.0000
536   Default   13.000000     deleted   13.000000   5.0000   5.0000
348   Default   25.000000         new   25.000000   4.0000   4.0000
859   Default   13.000000      Unable   13.000000   3.0000   3.0000
1068  Default   13.000000           _   13.000000   2.0000   2.0000
194   Default   13.000000         fix   13.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
690    Topic2   13.374968     version   14.560331   0.8465  -5.6194
344    Topic2    5.196354    annoying    5.662669   0.8454  -6.5649
357    Topic2   13.927483      update   15.381301   0.8321  -5.5790
11     Topic2  145.603668         app  169.675369   0.7784  -3.2319
49     Topic2   61.131752     booking   70.597397   0.7874  -4.0998
633    Topic2   15.930057     message   17.822369   0.8191  -5.4446
756    Topic2   13.804161     feature   15.389107   0.8227  -5.5878
155    Topic2   25.741201        book   30.040474   0.7769  -4.9647
220    Topic2   17.623650        Taxi   20.282995   0.7908  -5.3436
4      Topic2   47.995461         cab   65.437401   0.6214  -4.3417
348    Topic2   19.865040         new   25.276793   0.6905  -5.2239
131    Topic2   26.123091        fare   36.007938   0.6105  -4.9500
42     Topic2   36.902164         use   57.580208   0.4865  -4.6045
149    Topic2   30.182587        even   45.166378   0.5283  -4.8056
192    Topic2   13.651680      Please   17.110661   0.7055  -5.5990
253    Topic2   16.562716        keep   22.062061   0.6447  -5.4057
56     Topic2   73.329628        taxi  171.683105   0.0807  -3.9179
1507   Topic2   17.219278     pricing   23.732391   0.6106  -5.3668
494    Topic2   17.371428       using   24.578711   0.5843  -5.3580
25     Topic2   24.796711        work   47.219246   0.2873  -5.0021
22     Topic2   32.113678        time   86.985626  -0.0651  -4.7435
290    Topic2   29.583488         get   75.162521  -0.0011  -4.8256
1039   Topic2   26.39

# Topic Modelling (Positive)

In [25]:
pos_dict = corpora.Dictionary(pos_list_clean)
doc_term_matrix = [neg_dict.doc2bow(doc) for doc in pos_list_clean]

print(doc_term_matrix[0:10])

[[(68, 1), (112, 1), (120, 1), (192, 1), (199, 1), (241, 1), (243, 2), (290, 1), (322, 1), (494, 1)], [(1368, 1)], [(153, 1), (908, 1)], [(11, 1), (17, 1), (42, 1), (237, 1), (254, 1), (859, 1)], [(42, 1)], [(11, 1), (908, 1)], [(42, 1)], [(11, 1), (22, 1), (76, 1), (85, 1), (325, 1), (695, 1), (849, 1), (932, 1)], [(11, 1), (55, 1), (368, 1), (626, 1), (1126, 1)], [(908, 1)]]


In [20]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#doc_term_matrix = frequency of terms of all documents
#dictionary = all unique terms
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word=pos_dict, passes=50)

In [21]:
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, pos_dict)

D:\Windows\88B90FF3D0D3454AAACE9B026E2829E3.TMP\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.158287  0.010387       1        1  69.416328
0     -0.065064 -0.092778       2        1  16.068508
1     -0.093223  0.082391       3        1  14.515166, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
11    Default  139.000000  satisfactory  139.000000  30.0000  30.0000
42    Default   45.000000    ridiculous   45.000000  29.0000  29.0000
459   Default  153.000000          done  153.000000  28.0000  28.0000
1039  Default   47.000000          case   47.000000  27.0000  27.0000
241   Default  139.000000         Keeps  139.000000  26.0000  26.0000
306   Default   26.000000        lately   26.000000  25.0000  25.0000
255   Default   25.000000        refuse   25.000000  24.0000  24.0000
25    Default   39.000000          easy   39.000000  23.0000  23.0000
908   Default   13.000000         fancy   13.000000  22.0000  22.0000
365   Default   18.000000           tee   18.000000  21.0000  21.0000
1353  Default   21.000000         voted   21.000000  20.0000  20.0000
210   Default   13.000000    unfriendly   13.000000  19.0000  19.0000
149   Default   39.000000       feature   39.000000  18.0000  18.0000
292   Default   35.000000      straight   35.000000  17.0000  17.0000
296   Default   17.000000          slow   17.000000  16.0000  16.0000
1689  Default   10.000000        Readme   10.000000  15.0000  15.0000
333   Default   10.000000   Integration   10.000000  14.0000  14.0000
690   Default    9.000000          half    9.000000  13.0000  13.0000
1829  Default   12.000000          Lion   12.000000  12.0000  12.0000
194   Default    9.000000          Also    9.000000  11.0000  11.0000
859   Default    9.000000      Thompson    9.000000  10.0000  10.0000
120   Default   58.000000       invalid   58.000000   9.0000   9.0000
1424  Default   11.000000        Isetan   11.000000   8.0000   8.0000
237   Default    8.000000      password    8.000000   7.0000   7.0000
49    Default   66.000000       address   66.000000   6.0000   6.0000
1126  Default    8.000000      Grabtaxi    8.000000   5.0000   5.0000
101   Default   15.000000         error   15.000000   4.0000   4.0000
80    Default   15.000000        listed   15.000000   3.0000   3.0000
488   Default   10.000000        master   10.000000   2.0000   2.0000
757   Default   20.000000    competitor   20.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
1004   Topic3    2.249997      appeared    2.749637   1.7294  -6.4033
312    Topic3    2.246372     afternoon    2.747421   1.7286  -6.4049
410    Topic3    5.458941    forgetting    6.980567   1.6841  -5.5170
210    Topic3    9.973396    unfriendly   13.184597   1.6508  -4.9143
307    Topic3    4.422282          push    5.927897   1.6370  -5.7276
488    Topic3    7.232355        master   10.491442   1.5580  -5.2357
1146   Topic3    4.633397      terrible    6.492594   1.5926  -5.6809
306    Topic3   14.772891        lately   26.943733   1.3290  -4.5214
50     Topic3    5.475874        always    8.304996   1.5135  -5.5139
525    Topic3    3.272460           Bec    4.334560   1.6489  -6.0287
308    Topic3    5.122479           way    7.706209   1.5216  -5.5806
11     Topic3   48.698936  satisfactory  139.452911   0.8779  -3.3286
315    Topic3    4.424650          Even    6.469757   1.5500  -5.7270
42     Topic3   19.345774    ridiculous   45.310867   1.0789  -4.2518
255    Topic3   11.675319        refuse   25.387352   1.1532  -4.7567
182    Topic3    4.218169          Cant    6.084530   1.5636  -5.7748
64     Topic3    5.649857          year   11.895107   1.1855  -5.4826
357    Topic3    5.404984   comfortable   10.955963   1.2234  -5.5269
149    Topic3    9.600809       feature   39.073448   0.5264  -4.9524
101    Topic3    6.166577         error   15.33